In [4]:
%pip install deepface mediapipe pillow

Defaulting to user installation because normal site-packages is not writeable
  Using cached deepface-0.0.95-py3-none-any.whl.metadata (35 kB)
  Using cached mediapipe-0.10.21-cp39-cp39-macosx_11_0_universal2.whl.metadata (9.9 kB)
  Using cached retina_face-0.0.17-py3-none-any.whl.metadata (10 kB)
Using cached deepface-0.0.95-py3-none-any.whl (128 kB)
Using cached mediapipe-0.10.21-cp39-cp39-macosx_11_0_universal2.whl (49.2 MB)
Using cached retina_face-0.0.17-py3-none-any.whl (25 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [deepface]]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
from deepface import DeepFace
import mediapipe as mp
from PIL import Image
print("Libraries installed successfully!")

25-10-10 14:04:15 - Directory /Users/roopaldixit/.deepface has been created
25-10-10 14:04:15 - Directory /Users/roopaldixit/.deepface/weights has been created
Libraries installed successfully!


In [2]:
from deepface import DeepFace
import mediapipe as mp
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import os

In [3]:
# Initialize Mediapipe Face Detection
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5)

I0000 00:00:1760130776.193300 2062089 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M2


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1760130776.216703 2062713 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [7]:
# Step 1: Load known faces
known_faces_dir = "known_faces"
known_face_paths = [
    os.path.join(known_faces_dir, f) for f in os.listdir(known_faces_dir) if f.endswith((".jpeg", ".png"))
]
known_face_names = [os.path.splitext(os.path.basename(f))[0] for f in known_face_paths]

print("Known faces loaded:", known_face_names)

Known faces loaded: ['actor3', 'actor1']


In [10]:
# Step 2: Load test image
test_image_path = "unknown_faces/test1.jpeg"
image = Image.open(test_image_path).convert("RGB")
image_np = np.array(image)  # Convert to NumPy array for Mediapipe

In [11]:
# Step 3: Detect faces using Mediapipe
results = face_detection.process(image_np)
if not results.detections:
    print("No faces detected in the test image.")
    exit()

In [12]:
# Step 4: Initialize drawing on the image
draw = ImageDraw.Draw(image)
try:
    font = ImageFont.truetype("arial.ttf", 20)  # Try to use a font (download if needed)
except:
    font = ImageFont.load_default()  # Fallback to default font

In [13]:
# Step 5: Recognize faces using DeepFace
for detection in results.detections:
    # Get bounding box coordinates
    bbox = detection.location_data.relative_bounding_box
    h, w, _ = image_np.shape
    x_min = int(bbox.xmin * w)
    y_min = int(bbox.ymin * h)
    width = int(bbox.width * w)
    height = int(bbox.height * h)
    x_max = x_min + width
    y_max = y_min + height

    # Crop the face region for DeepFace
    face_region = image_np[y_min:y_max, x_min:x_max]
    if face_region.size == 0:
        continue  # Skip if the face region is empty

    # Save the face region temporarily for DeepFace
    temp_face_path = "temp_face.jpg"
    Image.fromarray(face_region).save(temp_face_path)

    # Compare with known faces
    name = "Unknown"
    for known_face_path, known_name in zip(known_face_paths, known_face_names):
        try:
            result = DeepFace.verify(temp_face_path, known_face_path, model_name="Facenet", enforce_detection=False)
            if result["verified"]:
                name = known_name
                break
        except:
            continue  # Skip if DeepFace fails to process the image

    # Draw bounding box and name
    draw.rectangle((x_min, y_min, x_max, y_max), outline="green", width=2)
    draw.text((x_min, y_min - 30), name, fill="green", font=font)

25-10-10 14:16:04 - 🔗 facenet_weights.h5 will be downloaded from https://github.com/serengil/deepface_models/releases/download/v1.0/facenet_weights.h5 to /Users/roopaldixit/.deepface/weights/facenet_weights.h5...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facenet_weights.h5
To: /Users/roopaldixit/.deepface/weights/facenet_weights.h5
100%|██████████| 92.2M/92.2M [00:02<00:00, 37.1MB/s]


In [14]:
# Step 6: Save and display the result
output_path = "output.jpg"
image.save(output_path)
image.show()  # Opens the image in the default image viewer

print(f"Output saved as {output_path}")

Output saved as output.jpg


In [15]:
test_image_path = "unknown_faces/test2.jpeg"
image = Image.open(test_image_path).convert("RGB")
image_np = np.array(image)

In [17]:
# Step 3: Detect faces using Mediapipe
results2 = face_detection.process(image_np)
if not results2.detections:
    print("No faces detected in the test image.")
    exit()

In [18]:
# Step 4: Initialize drawing on the image
draw = ImageDraw.Draw(image)
try:
    font = ImageFont.truetype("arial.ttf", 20)  # Try to use a font (download if needed)
except:
    font = ImageFont.load_default()  # Fallback to default font

In [19]:
# Step 5: Recognize faces using DeepFace
for detection in results2.detections:
    # Get bounding box coordinates
    bbox = detection.location_data.relative_bounding_box
    h, w, _ = image_np.shape
    x_min = int(bbox.xmin * w)
    y_min = int(bbox.ymin * h)
    width = int(bbox.width * w)
    height = int(bbox.height * h)
    x_max = x_min + width
    y_max = y_min + height

    # Crop the face region for DeepFace
    face_region = image_np[y_min:y_max, x_min:x_max]
    if face_region.size == 0:
        continue  # Skip if the face region is empty

    # Save the face region temporarily for DeepFace
    temp_face_path = "temp_face.jpg"
    Image.fromarray(face_region).save(temp_face_path)

    # Compare with known faces
    name = "Unknown"
    for known_face_path, known_name in zip(known_face_paths, known_face_names):
        try:
            result = DeepFace.verify(temp_face_path, known_face_path, model_name="Facenet", enforce_detection=False)
            if result["verified"]:
                name = known_name
                break
        except:
            continue  # Skip if DeepFace fails to process the image

    # Draw bounding box and name
    draw.rectangle((x_min, y_min, x_max, y_max), outline="green", width=2)
    draw.text((x_min, y_min - 30), name, fill="green", font=font)

In [20]:
# Step 6: Save and display the result
output_path = "output.jpg"
image.save(output_path)
image.show()  # Opens the image in the default image viewer

print(f"Output saved as {output_path}")

Output saved as output.jpg
